<img src="logos/Logo_operations.png" width=500></img>


## Welcome!
Welcome to the opperations group! you are responsible for deploy the code and models for your team and this guide will walk you through how to do you job. You will:

1. Set Up Code
1. Install requirements
1. configure your deployment environment
1. launch asset bucket
1. launch resources
1. Train and deploy models
1. register your team
1. retrain and redeploy models

## SetUp Notebook Instance
To launch your sagemaker notebook instance click the following link that will launch a cloudformation template that creates all the needed resources
https://console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/new?stackName=OperationsNotebook&templateURL=http://s3.amazonaws.com/jmc-website/ml-gameday/templates/operations-notebook.json

when the stack has completed go to its outputs and click the "JuypterURL" link to login to the instance

## Set up Code
1. Fork this (gameday bootstrap) repository
2. clone into your notebook instance by opening up a terminal
```shell
cd /home/ec2-user/SageMaker 
git clone https://github.com/aws-samples/ml-gameday-tutorial.git
```
3. change remote origin to point to your codecommit repo (you dont want your code in github because other teams can see that)
```shell
cd ml-gameday-tutorial
git remote set-url origin <code commit HTTPS Url>
```

4. create the following branches: dev, layout,shoot
```shell
git branch dev
git branch layout
git branch shoot
git push --all -u
```
5. share the following link with the other groups to create their notebook instance. It will automatically load your git repository into the instance (change the "ml-gameday" part of the link if you have your codecommit repository a different name)

https://console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/create/review?stackName=MLGameDayNotebook&templateURL=http://s3.amazonaws.com/jmc-website/ml-gameday/templates/notebook.json&param_CodeCommitRepoName=ml-gameday


## Install requirements

The follow cell will install all the needed nodejs requirements for the repository. If you do not have nodejs install you will need to install it

In [ ]:
!cd .. && npm install

The following cell will install the needed python requirements

In [ ]:
!cd .. && pip install -r requirements.txt

## Configure 
Next we need to configure the launch settings. do the following:

1. copy config.js
```shell
cp config.js.example config.js
```
2. configure config.js by editing the following fields

|Field|Meaning|Suggested Value|
|---|---|---|
|namespace|the name prepended to cloudformation templates. change this field to create dev and prod stacks|prod|
|region|The AWS region you want to launch resources into|us-east-1 or us-west-2|
|parameters.MasterAccount|The AWS Account where the GameDay engine runs|Given to you by GameDay coordinators|
|paremeters.GameArchiveBucket|The AWS S3 Bucket that contains the historical game data|Given to you by GameDay coordinators|

so you config.json might look like this
```json
{
    "namespace":"prod",
    "project":"battleship-bootstrap",
    "profile":"default",
    "region":"us-east-1",
    "parameters":{
        "MasterAccount":"123456789012",
        "GameArchiveBucket":"<gameday-archive-bucket>"
    }
}
```
3. Make sure you are on the master branch of the repo by runing the following command:

In [ ]:
!git branch

## Luanch Asset Bucket 
Now we are going to launch a cloudformation template that creates and S3 "Asset" bucket to hold all the assets we need to launch the main cloudformatio template. All you need to do is run the following cell

In [ ]:
!npm run bootstrap

## Launch

Lastly, we will launch the main cloudformation template with all the infastructure we will need. The following cell will launch a cloudformation template that has:

- aws-sagemaker-build pipeline for the Layout Endpoint
- aws-sagemaker-build pipeline for the Shoot Endpoint
- an SNS Topic and SQS Queue to recieve errors from the Game Engine
- A IAM role that the Game Engine will assume inorder to call your endpoint
- An AWS glue crawler and database for the game data

In [ ]:
!npm run up

## Train and Deploy Initial Models

now that you have the infastructure up you need to train and deploy your models. This repository already has code to deploy very simple Layout and Shoot endoint. the following two cells wil train and deploy those models

In [ ]:
!cd .. && ./bin/startLayoutPipeline.js

In [ ]:
!cd .. && ./bin/startShootPipeline.js

## Register Team
Once your two endpoints and deployed and up you can now register your team and start playing. Run the following cell to get the information needed to register your team

In [4]:
!../bin/register_outputs.sh

(node:44376) UnhandledPromiseRejectionWarning: Unhandled promise rejection (rejection id: 1): ValidationError: Stack with id battleship-bootstrap-dev-0 does not exist
(node:44376) [DEP0018] DeprecationWarning: Unhandled promise rejections are deprecated. In the future, promise rejections that are not handled will terminate the Node.js process with a non-zero exit code.


## Share Data Bucket
there is a single s3 bucket that data will need to be uploaded to inorder to be used by endpoint teams. You can find this bucket by running:

In [ ]:
!echo "Layout Data Bucket:"
!../templates/main/bin/output.js | jq '.LayoutDataBucket'
!echo "Shoot Data Bucket:"
!../templates/main/bin/output.js | jq '.ShootDataBucket'

## retrain and redeploy models

- Whent the Offensive and Defensive groups have changes they will commit them to their branches. Merge their branches into Dev first to check for any conflicts, then merge into master. 

- to update your models you can kick off new pipeline jobs using the same startLayoutPipeline.js and startShootPipeline.js scripts you ran earlier

- if you make changes to your infastructure (new resources or changes to aws-sagemaker-build parameters) run the following command

In [ ]:
!npm run update